calibrate overlap between human networks

In [9]:
import numpy as np
import pandas as pd
import calibration
import matplotlib.pyplot as plt
import networkx as nx
import math
import time
import copy
from itertools import combinations
import random
import os
from scipy import stats
import sys
from matplotlib import cm
from tqdm import trange
import seaborn as sns
import scipy


In [3]:
def sort_elist(elist,selfLink=False):
    '''
    selfLink: if False, the returned edgelist will not include self-interactions.
    '''
    if len(elist)==0 or type(elist[0][0])!=str: return list(set(sorted([tuple(sorted(x)) for x in elist])))

    if selfLink==False:
        res = list(set(sorted([tuple(sorted(pair)) for pair in elist if ('ENSG' in pair[0]) and ('ENSG' in pair[1]) and(pair[0]!=pair[1]) ])))
    else:
        res = list(set(sorted([tuple(sorted(pair)) for pair in elist if ('ENSG' in pair[0]) and ('ENSG' in pair[1]) ])))
    return res

# Import data

In [4]:
df_litbm = pd.read_csv('datasets/human/Lit-BM/Lit-BM_huri.tsv',sep='\t',header=None).astype(str)
litbmE = sort_elist(df_litbm.values)

df_bioplex = pd.read_csv('datasets/human/BioPlex/BioPlex3_processed.txt').astype(str)
bioplexE = sort_elist(df_bioplex.values)

df_qubic = pd.read_csv('datasets/human/Qubic/Qubic_processed.txt').astype(str)
qubicE = sort_elist(df_qubic.values)

df_huri = pd.read_csv('datasets/human/HuRI/[original]HuRI.tsv',sep='\t').astype(str)
huriE = sort_elist(df_huri.values)

df_huriPSN = pd.read_csv('datasets/human/HuRI-PSN/HuRI-PSN02_processed.txt').astype(str)
huriPSNE = sort_elist(df_huriPSN.values)
df_ccsb = pd.read_csv('datasets/human/CCSB/Supplementary Table 11.txt',sep='\t')

df_rual = df_ccsb[df_ccsb.in_Rual==1].iloc[:,:2].astype(str)
rualE = sort_elist(df_rual.values)

df_ven = df_ccsb[df_ccsb.in_Venkatesan==1].iloc[:,:2].astype(str)
venE = sort_elist(df_ven.values)

df_yu = df_ccsb[df_ccsb.in_Yu==1].iloc[:,:2].astype(str)
yuE = sort_elist(df_yu.values)

df_rolland = df_ccsb[df_ccsb.in_Rolland==1].iloc[:,:2].astype(str)
rollandE = sort_elist(df_rolland.values)

df_yang = df_ccsb[df_ccsb.in_Yang==1].iloc[:,:2].astype(str)
yangE = sort_elist(df_yang.values)

df_i3d = pd.read_csv('datasets/human/I3D/I3D_processed.txt')
i3dE = sort_elist(df_i3d.values)

df_string900 = pd.read_csv('datasets/human/STRING/STRING_physical_threshold900.txt')
string900E = sort_elist(df_string900.values)

In [7]:
# check nodes and links
test = huriPSNE
print("node: %s, links: %s"%(len(set(np.array(test).flatten())),len(test)))

node: 4229, links: 33125


# All by all network of networks

In [8]:
labels = ['HuRI','HI-I-05','Venkatesan-09','HI-II-14','Yang-16','Yu-11','Lit-BM-17','STRING(HsC)-H','I3D-H','BioPlex','Qubic','HuRI-PSN']
dictElist = {'HuRI':huriE,'HI-I-05':rualE,'Venkatesan-09':venE,'HI-II-14':rollandE,'Yang-16':yangE,'Yu-11':yuE,
             'Lit-BM-17':litbmE,'STRING(HsC)-H':string900E,'I3D-H':i3dE,
             'BioPlex':bioplexE,'Qubic':qubicE,
             'HuRI-PSN':huriPSNE}

## Statistics

In [10]:
%%time
res = []
for label in labels:
    test = dictElist[label]
    res.append([label,len(set(np.array(test).flatten())),len(test)])
df_stat = pd.DataFrame(res,columns=['dataset','nodes','links'])
n = df_stat['nodes']
l = df_stat['links']
density = l/(n*(n-1)/2)
df_stat['density'] = density
df_stat.to_csv('results human/dataset_overview.txt',index=False)
df_stat

CPU times: user 389 ms, sys: 10.5 ms, total: 399 ms
Wall time: 401 ms


,dataset,nodes,links,density
0,HuRI,8245,52067,0.001532
1,Rual,1500,2561,0.002278
2,Venkatesan,195,187,0.009886
3,Rolland,4111,13118,0.001553
4,Yang,502,717,0.005702
5,Yu,1133,1127,0.001757
6,Lit-BM-17,5956,12758,0.000719
7,STRING(HsC)-H,6662,39567,0.001783
8,I3D-H,3155,4354,0.000875
9,BioPlex,12408,97078,0.001261


## All by all - optimize pos

In [10]:
df_preRes = pd.DataFrame()
testedPairs = [(row[0],row[1]) for row in df_preRes.values]+[(row[1],row[0]) for row in df_preRes.values]
allPairs = list(combinations(labels,2))
results = []
s = time.time()
for i in trange(len(allPairs)):
    pair = allPairs[i]
    if pair in testedPairs: continue # if the pair has been calculated, skip it.
    pos1_mean, pos1_sigma, cur_iter1 = calibration.random_subnetwork.optimize_pos(dictElist[pair[0]],dictElist[pair[1]])
    pos2_mean, pos2_sigma, cur_iter2 = calibration.random_subnetwork.optimize_pos(dictElist[pair[1]],dictElist[pair[0]])
    results.append([pair[0],pair[1],pos1_mean, pos1_sigma, cur_iter1, pos2_mean, pos2_sigma, cur_iter2])

100%|██████████| 66/66 [2:19:39<00:00, 126.96s/it]  


In [ ]:
df_results = pd.DataFrame(results,columns=['A','B','pos1_mean', 'pos1_sigma', 'cur_iter1', 'pos2_mean', 'pos2_sigma', 'cur_iter2'])
df_results.to_csv('results human/all_by_all_pos.txt',float_format='%.2f')

## All by all - optimize neg

In [9]:
df_preRes = pd.DataFrame()
testedPairs = [(row[0],row[1]) for row in df_preRes.values]+[(row[1],row[0]) for row in df_preRes.values]
allPairs = list(combinations(labels,2))
results = []
s = time.time()
for i in trange(len(allPairs)):
    pair = allPairs[i]
    if pair in testedPairs: continue # if the pair has been calculated, skip it.
    neg1_mean, neg1_sigma, cur_iter1 = calibration.random_network.optimize_neg(dictElist[pair[0]],dictElist[pair[1]])
    neg2_mean, neg2_sigma, cur_iter2 = calibration.random_network.optimize_neg(dictElist[pair[1]],dictElist[pair[0]])
    results.append([pair[0],pair[1],neg1_mean, neg1_sigma, cur_iter1, neg2_mean, neg2_sigma, cur_iter2])

100%|██████████| 66/66 [36:57<00:00, 33.60s/it] 


In [10]:
df_results = pd.DataFrame(results,columns=['A','B','neg1_mean', 'neg1_sigma', 'cur_iter1', 'neg2_mean', 'neg2_sigma', 'cur_iter2'])
df_results.to_csv('results human/all_by_all_neg.txt',float_format='%.2f')

## Combine results

In [ ]:
import scipy
df_pos = pd.read_csv('results/all_by_all_pos.txt')
df_neg = pd.read_csv('results/all_by_all_neg.txt')
df = pd.merge(df_pos,df_neg,on=['A','B'])

# select neg benchmark
z_neg1 = [abs((obs-neg1_mean)/neg1_sigma) if neg1_sigma != 0 else np.nan for obs,
          neg1_mean, neg1_sigma in zip(df['obs'], df['neg1_mean'], df['neg1_sigma'])]
z_neg2 = [abs((obs-neg2_mean)/neg2_sigma) if neg2_sigma != 0 else np.nan for obs,
          neg2_mean, neg2_sigma in zip(df['obs'], df['neg2_mean'], df['neg2_sigma'])]
df['selected_neg'] = [neg1 if z1 < z2 else neg2 for z1, z2, neg1,
                      neg2 in zip(z_neg1, z_neg2, df['neg1_mean'], df['neg2_mean'])]
df['selected_neg_sigma'] = [neg1 if z1 < z2 else neg2 for z1, z2, neg1,
                            neg2 in zip(z_neg1, z_neg2, df['neg1_sigma'], df['neg2_sigma'])]

# select pos benchmark
z_pos1 = [abs((obs-pos1_mean)/pos1_sigma) if pos1_sigma != 0 else np.nan for obs,
          pos1_mean, pos1_sigma in zip(df['obs'], df['pos1_mean'], df['pos1_sigma'])]
z_pos2 = [abs((obs-pos2_mean)/pos2_sigma) if pos2_sigma != 0 else np.nan for obs,
          pos2_mean, pos2_sigma in zip(df['obs'], df['pos2_mean'], df['pos2_sigma'])]
df['selected_pos'] = [pos1 if z1 < z2 else pos2 for z1, z2, pos1,
                      pos2 in zip(z_pos1, z_pos2, df['pos1_mean'], df['pos2_mean'])]
df['selected_pos_sigma'] = [pos1 if z1 < z2 else pos2 for z1, z2, pos1,
                            pos2 in zip(z_pos1, z_pos2, df['pos1_sigma'], df['pos2_sigma'])]

score,score_sigma = calibration.cal_score(df['obs'],df['selected_neg'],df['selected_neg_sigma'],df['selected_pos'],df['selected_pos_sigma'])
df['score'] = score
df['score_sigma'] = score_sigma
neg_z = [(obs-neg_mean)/neg_sigma if neg_sigma!=0 else np.nan for obs,neg_mean,neg_sigma in zip(df['obs'],df['selected_neg'],df['selected_neg_sigma'])]
df['neg_p'] = scipy.stats.norm.sf(neg_z)
pos_z = [(obs-pos_mean)/pos_sigma if pos_sigma!=0 else np.nan for obs,pos_mean,pos_sigma in zip(df['obs'],df['selected_pos'],df['selected_pos_sigma'])]
df['pos_p'] = 1-scipy.stats.norm.sf(pos_z)
df['pos_p_binary'] = [1 if (pos_p>=0.05 and neg_p<0.05) else 0 for neg_p,pos_p in zip(df['neg_p'],df['pos_p'])]
df.round(3).to_csv("results human/all_by_all_networks_no_instance.txt",index=False)